### Integrated Gradients
https://captum.ai/docs/attribution_algorithms#integrated-gradients

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
from captum.attr import IntegratedGradients

# Define a GRU-based model
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(1, batch_size, self.hidden_size)
        out, _ = self.gru(x, h0)
        out = F.relu(out)
        out = self.fc(out)
        return out

# Define a function to compute IntegratedGradients
def compute_ig(model, x, baseline):
    ig = IntegratedGradients(model)
    out = model(x)
    attributions, delta = ig.attribute(x, baseline, return_convergence_delta=True)
    return attributions, delta

# Example usage
model = GRUModel(input_size=10, hidden_size=5, output_size=2)
x = torch.randn(3, 5, 10)  # input sequence with batch size=3, sequence length=5, and input size=10
baseline = torch.zeros_like(x)  # baseline tensor with the same shape as the input
assert baseline.shape == x.shape, "Baseline tensor must have the same shape as the input tensor"
target = torch.tensor([1])  # target class

# Compute IntegratedGradients for the input sequence
ig, delta = compute_ig(model, x, baseline)

# Print the IntegratedGradients scores for each element in the input sequence
print("IntegratedGradients scores:", ig.sum(dim=2))
print("Convergence delta:", delta)


### Gradient SHAP
https://captum.ai/docs/attribution_algorithms#gradient-shap

In [ ]:
import torch
import torch.nn as nn

class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(GRU, self).__init__()
        self.gru = nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        _, h_n = self.gru(x)
        out = self.fc(h_n.squeeze(0))
        return out
from captum.attr import GradientShap
import numpy as np

# assume we have a trained GRU model and a test input tensor `x_test`
model = GRU(input_size=10, hidden_size=20, num_layers=2, output_size=2)
model.eval()
x_test = torch.randn(1, 10)

# define a baseline tensor to compare input to
baseline = torch.zeros_like(x_test)

# create a GradientShap object for the model
grad_shap = GradientShap(model)

# compute the attribution scores using Gradient SHAP
attributions = grad_shap.attribute(x_test, baseline, n_samples=50)

# print the importance scores for each input feature
print('Importance scores:', attributions.squeeze(0).detach().numpy())


### Deep Lift SHAP
https://captum.ai/docs/attribution_algorithms#deeplift-shap

In [ ]:
from captum.attr import DeepLiftShap

# assume we have a trained GRU model and a test input tensor `x_test`
model = GRU(input_size=10, hidden_size=20, num_layers=2, output_size=2)
model.eval()
x_test = torch.randn(1, 10)

# define a baseline tensor to compare input to
baseline = torch.zeros_like(x_test)

# create a DeepLiftShap object for the model
deep_lift_shap = DeepLiftShap(model)

# compute the attribution scores using DeepLift SHAP
attributions = deep_lift_shap.attribute(x_test, baseline)

# print the importance scores for each input feature
print('Importance scores:', attributions.squeeze(0).detach().numpy())


### Saliency
https://captum.ai/docs/attribution_algorithms#saliency

In [ ]:
from captum.attr import Saliency

# assume we have a trained GRU model and a test input tensor `x_test`
model = GRU(input_size=10, hidden_size=20, num_layers=2, output_size=2)
model.eval()
x_test = torch.randn(1, 10)

# create a Saliency object for the model
saliency = Saliency(model)

# compute the attribution scores using saliency
attributions = saliency.attribute(x_test)

# print the importance scores for each input feature
print('Importance scores:', attributions.squeeze(0).detach().numpy())


### Occlusion
https://captum.ai/docs/attribution_algorithms#occlusion

In [ ]:
from captum.attr import Occlusion

# assume we have a trained GRU model and a test input tensor `x_test`
model = GRU(input_size=10, hidden_size=20, num_layers=2, output_size=2)
model.eval()
x_test = torch.randn(1, 10)

# define the sequence length and step size for occlusion
sequence_length = 3
step_size = 1

# create an Occlusion object for the model
occlusion = Occlusion(model)

# compute the attribution scores using occlusion
attributions = occlusion.attribute(x_test, sliding_window_shapes=(sequence_length,), strides=(step_size,))

# print the importance scores for each input feature
print('Importance scores:', attributions.squeeze(0).detach().numpy())


In [2]:
import torch
import torch.nn as nn
from captum.attr import IntegratedGradients

class GRU4Rec(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(GRU4Rec, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        
        # define the GRU layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        
        # define the output layer
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        # input shape: (batch_size, seq_length, input_size)
        batch_size = x.size(0)
        
        # initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        
        # pass input through GRU layer
        out, _ = self.gru(x, h0)
        
        # reshape output from (batch_size, seq_length, hidden_size) to (batch_size*seq_length, hidden_size)
        out = out.contiguous().view(-1, self.hidden_size)
        
        # pass output through output layer
        out = self.fc(out)
        
        # reshape output back to (batch_size, seq_length, output_size)
        out = out.view(batch_size, -1, self.output_size)
        
        # return output
        return out

# create an instance of the GRU4Rec model
model = GRU4Rec(input_size=10, hidden_size=32, output_size=2)

# create a sequence of inputs to explain
inputs = torch.randn(1, 10, 10)

# create an instance of the IntegratedGradients class
ig = IntegratedGradients(model)

# compute the attribution scores using integrated gradients
attribution_scores = ig.attribute(inputs)


AssertionError: Target not provided when necessary, cannot take gradient with respect to multiple outputs.